In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
data = pd.read_csv('../input/voice.csv')
data['label'] = data['label'].map({'female': 1, 'male': 0})

In [ ]:
import random
data = data.ix[random.sample(list(data.index), len(data))]

In [ ]:
proc = 80
proc_test = 10
test_count = int((proc / 100) * len(data))
cross_count = int(((proc + proc_test) / 100) * len(data))
data_train = data[:test_count]

In [ ]:
data_train.corr()['label']

In [ ]:
parametr = ['meanfreq', 'sd', 'median', 'Q25', 'IQR', 'sp.ent', 'sfm', 'mode', 'centroid', 'meanfun', \
           'minfun', 'maxfun', 'meandom', 'mindom', 'maxdom', 'dfrange']
X = np.array(data.ix[:, parametr])
y = np.array(data['label'])
X_train = np.array(data_train.ix[:, parametr])
y_train = np.array(data_train['label'])
X_test = X[test_count:cross_count, :]
y_test = y[test_count:cross_count]
X_cross = X[cross_count:, :]
y_cross = y[cross_count:]

In [ ]:
<h6> Choose parameter with correlation > 0.1 </h6>

In [ ]:
 Choose parameter with correlation > 0.1

In [ ]:
import numpy as np

def sigmoid(x):
    return 1.0 / (1.0 + np.exp( - x))

def sigmoid_prime(x):
    return sigmoid(x) * (1.0 - sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanh_prime(x):
    return 1.0 - x ** 2


class NeuralNetwork:

    def __init__(self, layers, activation = 'tanh'):
        if activation == 'sigmoid':
            self.activation = sigmoid
            self.activation_prime = sigmoid_prime
        elif activation == 'tanh':
            self.activation = tanh
            self.activation_prime = tanh_prime

        # Set weights
        self.weights = []
        # layers = [2,2,1]
        # range of weight values (-1,1)
        # input and hidden layers - random((2+1, 2+1)) : 3 x 3
        for i in range(1, len(layers) - 1):
            r = 2 * np.random.random((layers[i - 1] + 1, layers[i] + 1)) -1
            self.weights.append(r)
        # output layer - random((2+1, 1)) : 3 x 1
        r = 2 * np.random.random( (layers[i] + 1, layers[i + 1])) - 1
        self.weights.append(r)

    def fit(self, X, y, learning_rate = 0.01, epochs=1000):
        
        ones = np.atleast_2d(np.ones(X.shape[0]))
        X = np.concatenate((ones.T, X), axis=1)
         
        for k in range(epochs):
            for i in range(X.shape[0]):
                a = [X[i]]

                for l in range(len(self.weights)):
                    dot_value = np.dot(a[l], self.weights[l])
                    activation = self.activation(dot_value)
                    a.append(activation)
            # output layer
                error = y[i] - a[-1]
                deltas = [error * self.activation_prime(a[-1])]

                for l in range(len(a) - 2, 0, -1): 
                    deltas.append(deltas[-1].dot(self.weights[l].T) * self.activation_prime(a[l]))
                deltas.reverse()

            # backpropagation
                for i in range(len(self.weights)):
                    layer = np.atleast_2d(a[i])
                    delta = np.atleast_2d(deltas[i])
                    self.weights[i] += learning_rate * layer.T.dot(delta)

    def predict(self, x): 
        a = np.concatenate((np.ones(1).T, np.array(x)), axis= 0)      
        for l in range(0, len(self.weights)):
            a = self.activation(np.dot(a, self.weights[l]))
        return a

In [ ]:
from sklearn import metrics
nn = NeuralNetwork([16, 10, 1])
nn.fit(X_train, y_train, 0.01, 1000)
predict = np.zeros((len(y_test), 1))
for i in range(len(X_test)):
    predict[i] = nn.predict(X_test[i])
fpr, tpr, thresholds = metrics.roc_curve(y_test.ravel(), predict.ravel())
print('AUC = ', metrics.auc(fpr, tpr))
print('GINI = ', 2 * metrics.auc(fpr, tpr) - 1, '\n')
pred = predict > 0.5
print('Accuracy= ', 1 - np.sum(np.abs(y_test.ravel() - pred.ravel())) / y_test.shape[0])
import matplotlib.pyplot as plt

plt.plot([0, 1], [0, 1], color='navy')
plt.plot(fpr, tpr, color='darkorange')
plt.show()

In [ ]:
proc = 80
proc_test = 10
test_count = int((proc / 100) * len(data))

In [ ]:
import copy 
nn_weights = copy.deepcopy(nn.weights)

In [ ]:
nn.weights = copy.deepcopy(nn_weights)
nn.fit(X_train, y_train, 0.001, 1000)
predict = np.zeros((len(y_test), 1))
for i in range(len(X_test)):
    predict[i] = nn.predict(X_test[i])
fpr, tpr, thresholds = metrics.roc_curve(y_test.ravel(), predict.ravel())
print('AUC = ', metrics.auc(fpr, tpr))
print('GINI = ', 2 * metrics.auc(fpr, tpr) - 1, '\n')
pred = predict > 0.5
print('Accuracy= ', 1 - np.sum(np.abs(y_test.ravel() - pred.ravel())) / y_test.shape[0])
import matplotlib.pyplot as plt

plt.plot([0, 1], [0, 1], color='navy')
plt.plot(fpr, tpr, color='darkorange')
plt.show()

In [ ]:
import copy 
nn_weights = copy.deepcopy(nn.weights)

In [ ]:
nn.weights = copy.deepcopy(nn_weights)
nn.fit(X_train, y_train, 0.001, 1000)
predict = np.zeros((len(y_test), 1))
for i in range(len(X_test)):
    predict[i] = nn.predict(X_test[i])
fpr, tpr, thresholds = metrics.roc_curve(y_test.ravel(), predict.ravel())
print('AUC = ', metrics.auc(fpr, tpr))
print('GINI = ', 2 * metrics.auc(fpr, tpr) - 1, '\n')
pred = predict > 0.5
print('Accuracy= ', 1 - np.sum(np.abs(y_test.ravel() - pred.ravel())) / y_test.shape[0])
import matplotlib.pyplot as plt

plt.plot([0, 1], [0, 1], color='navy')
plt.plot(fpr, tpr, color='darkorange')
plt.show()

In [ ]:
import copy 
nn_weights = copy.deepcopy(nn.weights)

In [ ]:
nn.weights = copy.deepcopy(nn_weights)
predict = np.zeros((len(y_test), 1))
for i in range(len(X_test)):
    predict[i] = nn.predict(X_test[i])
fpr, tpr, thresholds = metrics.roc_curve(y_test.ravel(), predict.ravel())
print('AUC = ', metrics.auc(fpr, tpr))
print('GINI = ', 2 * metrics.auc(fpr, tpr) - 1, '\n')
pred = predict > 0.5
print('Accuracy= ', 1 - np.sum(np.abs(y_test.ravel() - pred.ravel())) / y_test.shape[0])
import matplotlib.pyplot as plt

plt.plot([0, 1], [0, 1], color='navy')
plt.plot(fpr, tpr, color='darkorange')
plt.show()

In [ ]:
nn.weights = copy.deepcopy(nn_weights)
predict = np.zeros((len(y_cross), 1))
for i in range(len(X_cross)):
    predict[i] = nn.predict(X_cross[i])
fpr, tpr, thresholds = metrics.roc_curve(y_cross.ravel(), predict.ravel())
print('AUC = ', metrics.auc(fpr, tpr))
print('GINI = ', 2 * metrics.auc(fpr, tpr) - 1, '\n')
pred = predict > 0.5
print('Accuracy= ', 1 - np.sum(np.abs(y_cross.ravel() - pred.ravel())) / y_cross.shape[0])
import matplotlib.pyplot as plt

plt.plot([0, 1], [0, 1], color='navy')
plt.plot(fpr, tpr, color='darkorange')
plt.show()